Run with the following conda environment: `../../conda_envs/training_env`

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib import pyplot

from os import listdir

In [ ]:
results_folder = "../../results/simu_output/scheduling/synth/"
baseline_folder = "../../results/simu_output/scheduling/"

In [ ]:
plt.rc('font', size=16)
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.title_fontsize'] = 14
plt.rc('pdf', fonttype=42)

In [ ]:
coda_name = 'DUMBO'
simus = listdir(results_folder)

In [ ]:
results = []
for s in simus:
    s_list = s.split('_')
    tpr = float(s_list[4][:-3])
    tnr = float(s_list[5][:-7])
    df = pd.read_csv(f"{results_folder}{s}", names=["load", "avg_slowdown", "tmp1", "tmp2"], sep=" ")
    df = df[["load", "avg_slowdown"]]
    df["tpr"] = tpr
    df["tnr"] = tnr
    df["fnr"] = round(1.0 - df["tpr"], 5)
    df["fpr"] = round(1.0 - df["tnr"], 5)
    results.append(df)

results = pd.concat(results)
load_synth = results['load'].values[-1]

In [ ]:
nflows = "1M"
trace = "caida"

path_pfabric = f"{baseline_folder}result_pfabric_{trace}_{nflows}.txt"
path_fifo = f"{baseline_folder}result_fifo_{trace}_{nflows}.txt"

fct_pfabric = pd.read_csv(path_pfabric, names=["load", "avg_slowdown", "tmp1", "tmp2"], sep=" ")
fct_pfabric = fct_pfabric[fct_pfabric["load"] == load_synth]["avg_slowdown"].values[0]

fct_fifo = pd.read_csv(path_fifo, names=["load", "avg_slowdown", "tmp1", "tmp2"], sep=" ")
fct_fifo = fct_fifo[fct_fifo["load"] == load_synth]["avg_slowdown"].values[0]

In [ ]:
dumbo_fct = pd.read_csv(baseline_folder + "result_coda_"+ trace + "_" + nflows + ".txt", names=["load", "avg_slowdown", "tmp1", "tmp2"], sep=" ")
dumbo_fct = dumbo_fct[dumbo_fct["load"] == load_synth]["avg_slowdown"].values[0]
dumbo_fnr = pd.read_csv("../../results/5_pk/tcp_udp/initial5min_pruning+feat_selection+quantization_0dryrun_caida/rates.txt", names=["rate", "value"])
dumbo_fnr = dumbo_fnr[dumbo_fnr["rate"] == "FNR"]["value"].values[0]

In [ ]:
fig, ax_elephs = plt.subplots(1, 1, figsize=(6, 4))

ax2_elephs = ax_elephs.twiny()
ax_elephs.set_xlabel("Elephants mispredictions")
ax_elephs.set_ylabel("Normalized FCT")

ax_elephs.plot(results.sort_values("fnr")["fnr"], results.sort_values("fnr")["avg_slowdown"], color="tab:blue")
l_synth = mlines.Line2D([], [], color="tab:blue", marker='None', linestyle='-', label=f'hint-based (synth)')

# baselines
ax_elephs.hlines(y=fct_fifo, xmin=0, xmax=1, label="FIFO", color="black", ls="--")
ax_elephs.hlines(y=fct_pfabric, xmin=0, xmax=1, label="pFabric", color="black")

# DUMBO
ax_elephs.scatter(dumbo_fnr, dumbo_fct, marker="*", color='black', s=120, zorder=2)
l_coda = mlines.Line2D([], [], color='black', marker='*', linestyle='None', markersize=10, label=f'{coda_name}')

# Mice mispred equivalent
ax_elephs.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.], [0.0, 0.2, 0.4, 0.6, 0.8, 1.])
ax_elephs.set_yticks([1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7])
mice_tick_locations = ax_elephs.get_xticks()
def tick_function(x_list, df):
    ticks = []
    for v in x_list:
        mice_tick = df[df["fnr"] == v]["fpr"].values[0]
        ticks.append(round(mice_tick, 3))
    return ticks
ax2_elephs.set_xlim(ax_elephs.get_xlim())
ax2_elephs.set_xticks(mice_tick_locations)
ax2_elephs.set_xticklabels(tick_function(mice_tick_locations, results))
ax2_elephs.set_xlabel("Mice mispredictions")

pyplot.gca().add_artist(
    pyplot.legend(
        handles=[l_synth,
                 l_coda
                 ], ncol=2, columnspacing=0.9, frameon=False,
        loc='lower center', borderaxespad=0.2
    )
)

ax_elephs.legend(
    ncol=1, columnspacing=0.9, frameon=False, loc='upper left', title='Baselines', borderaxespad=1
)

ax_elephs.grid(alpha=0.5)
ax_elephs.set_xlim([0, 1])
ax_elephs.set_ylim([0, None])

print(f"{trace.upper()} - 20K Eleph Tr. - {nflows} Flows - Load {load_synth}")
plt.subplots_adjust(left=0.09, right=0.97, bottom=0.15, top=0.86)
plt.savefig(f'./out/scheduling_synth.pdf', format='pdf', dpi=1200)
plt.show()